## Tokenization using Byte-Pair Encoding and a Unigram Language Model

Author: Pierre Nugues with help from Marcus Klang

In this assignment, you will create a tokenization program to handle subwords. You will in parallel experiment with SentencePiece, a high performance subword tokenizer, used by Google, in llama (https://arxiv.org/pdf/2302.13971) and to build large training corpora (https://aclanthology.org/2020.lrec-1.494.pdf). 

## Description

In many scripts from Asia, like Chinese, Korean, or Japanese scripts, tokenization cannot rely on white spaces. The byte-pair encoding and the unigram language model are techniques that are now common in machine translation to carry out a tokenization at a subword level. Subword level tokenization shows better multilingual capabilities.

You will follow two papers: 
* _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ by Kudo (2018) (https://arxiv.org/pdf/1804.10959.pdf) and 
* _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020) (https://aclanthology.org/2020.findings-emnlp.414.pdf). 

You will use a small corpus make it easier to test and correct your code. Note also that you will use _characters_ and not _bytes_ in this lab as this is simpler to implement. For a complete program, see the link at the end.

## Modules

In [96]:
import regex as re
import tqdm as tqdm
import math
from collections import Counter

## Dataset

As dataset, you will first use a small corpus and then, if you have time, test your program on a larger one. Here we take the smallest novel from Selma Lagerlöf in our corpus: _En herrgårdssägen_

In [97]:
import os
from zipfile import ZipFile
import requests

# Parameters for Selma dataset
SELMA_URL = "https://github.com/pnugues/ilppp/raw/master/programs/corpus/Selma.zip"

SELMA_FILES = [
    os.path.join("Selma", fname)
    for fname in
    [
        "bannlyst.txt",
        "gosta.txt",
        "herrgard.txt",
        "jerusalem.txt",
        "kejsaren.txt",
        "marbacka.txt",
        "nils.txt",
        "osynliga.txt",
        "troll.txt"
    ]
]


def download_and_extract_selma():
    """Downloads and unpacks Selma.zip"""

    # Download if not all files exist
    req = requests.get(SELMA_URL, stream=True)
    if req.status_code != 200:
        print("Failed to download file, got status: " + req.status_code)
        req.close()
    else:
        with open("Selma.zip", "wb") as fd:
            written = 0
            for chunk in req.iter_content(chunk_size=65536):
                fd.write(chunk)
                written += len(chunk)
                print("Downloading: %d bytes written to Selma.zip" % written)

        print("Selma.zip donwnloaded.")
        req.close()

        selma_zipfile = ZipFile("Selma.zip")
        selma_files_to_extract = [zi for zi in selma_zipfile.filelist if not zi.filename.startswith(
            "__") and zi.filename.endswith(".txt")]
        for zi in selma_files_to_extract:
            selma_zipfile.extract(zi)
            print("Extracted: " + zi.filename)

        print("Done!")


# If not all path exists (all are true), then download
if not all([os.path.exists(fname) for fname in SELMA_FILES]):
    download_and_extract_selma()
else:
    print("Selma has been downloaded.")

SELMA_FILES

Selma has been downloaded.


['Selma/bannlyst.txt',
 'Selma/gosta.txt',
 'Selma/herrgard.txt',
 'Selma/jerusalem.txt',
 'Selma/kejsaren.txt',
 'Selma/marbacka.txt',
 'Selma/nils.txt',
 'Selma/osynliga.txt',
 'Selma/troll.txt']

In [98]:
# FILE_PATH = '../../corpus/Selma.txt'
FILE_PATH = 'Selma/herrgard.txt'

Read the corpus and store it in the `corpus` string variable.

In [99]:
with open(FILE_PATH, encoding='utf8') as f:
    corpus_raw = f.read().strip()

In [100]:
corpus_raw[:100]

'Selma Lagerlöf\nEn herrgårdssägen\n\n\n\nBokutgåva\n\n\nAlbert Bonniers förlag, Stockholm 1899.\n\n\n\nI.\n\n\nDet '

## Understanding SentencePiece
SentencePiece is a program that implements both BPE and a unigram language model. To be sure you understand how it works, you will first run this code.

This code comes from the first cells of a larger program: https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb
from Google.


In [101]:
%pip install sentencepiece
#!wget https: // raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt


Note: you may need to restart the kernel to use updated packages.


In [102]:
import sentencepiece as spm

SentencePiece implements BPE and unigram. The unigram method is the default. In this lab, we start with the BPE method with the parameter: `model_type`

We run SentencePiece on `herrgard.txt`. Do not modify the parameters to be able to compare the results to your implementation.

In [103]:
spm.SentencePieceTrainer.train(
    '--input=Selma/herrgard.txt --model_prefix=m --vocab_size=116 --model_type=BPE --user_defined_symbols=0,1,2,3,4,5,6,7,8,9 ')

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=Selma/herrgard.txt --model_prefix=m --vocab_size=116 --model_type=BPE --user_defined_symbols=0,1,2,3,4,5,6,7,8,9 
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: Selma/herrgard.txt
  input_format: 
  model_prefix: m
  model_type: BPE
  vocab_size: 116
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: 0
  user_defined_symbols: 1
  user_defined_symbols: 2
  user_defined_symbols: 3
  user_defined_symbols: 4
  user_defined_symbols: 5
  user_defined_symbols: 6
  u

Once you have trained your model, read `m.vocab` output, where you will find the list of subwords. Your program should produce something similar in the end.

In [104]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

We tokenize a string. This output consists of subword ids.

In [105]:
sp.encode('Selma Lagerlöf')

[63, 96, 64, 71, 76, 65, 63, 110, 65, 75, 37, 71, 84, 78]

In [106]:
sp.id_to_piece(63), sp.id_to_piece(96)

('▁', 'S')

In [107]:
[sp.id_to_piece(i) for i in range(30)]

['<unk>',
 '<s>',
 '</s>',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '▁s',
 'de',
 '▁h',
 'en',
 'an',
 'tt',
 'ar',
 '▁v',
 '▁f',
 '▁a',
 'om',
 'on',
 'll',
 '▁de',
 '▁m',
 'ör',
 '▁o']

We tokenize a string and get a list of subwords

In [108]:
sp.encode('Selma Lagerlöf', out_type=str)

['▁', 'S', 'e', 'l', 'm', 'a', '▁', 'L', 'a', 'g', 'er', 'l', 'ö', 'f']

In [109]:
sp.encode('123', out_type=str)

['▁', '1', '2', '3']

We tokenize the whole corpus

In [110]:
sp.encode(corpus_raw, out_type=str)

['▁',
 'S',
 'e',
 'l',
 'm',
 'a',
 '▁',
 'L',
 'a',
 'g',
 'er',
 'l',
 'ö',
 'f',
 '▁',
 'E',
 'n',
 '▁h',
 'er',
 'r',
 'g',
 'å',
 'r',
 'd',
 's',
 's',
 'ä',
 'g',
 'en',
 '▁',
 'B',
 'o',
 'k',
 'u',
 't',
 'g',
 'å',
 'v',
 'a',
 '▁',
 'A',
 'l',
 'b',
 'er',
 't',
 '▁',
 'B',
 'on',
 'n',
 'i',
 'er',
 's',
 '▁för',
 'l',
 'a',
 'g',
 ',',
 '▁',
 'S',
 't',
 'o',
 'ck',
 'h',
 'o',
 'l',
 'm',
 '▁',
 '1',
 '8',
 '9',
 '9',
 '.',
 '▁',
 'I',
 '.',
 '▁',
 'D',
 'et',
 '▁var',
 '▁en',
 '▁s',
 'k',
 'ö',
 'n',
 '▁h',
 'ö',
 'st',
 'd',
 'a',
 'g',
 '▁h',
 'än',
 'e',
 'm',
 'o',
 't',
 '▁s',
 'l',
 'u',
 't',
 'et',
 '▁a',
 'f',
 '▁t',
 'r',
 'e',
 'tt',
 'i',
 'o',
 't',
 'a',
 'l',
 'et',
 '.',
 '▁',
 'P',
 'å',
 '▁de',
 'n',
 '▁t',
 'i',
 'de',
 'n',
 '▁f',
 'an',
 'n',
 's',
 '▁i',
 '▁',
 'U',
 'p',
 's',
 'a',
 'l',
 'a',
 '▁e',
 'tt',
 '▁h',
 'ö',
 'g',
 't',
 ',',
 '▁g',
 'u',
 'l',
 't',
 '▁t',
 'v',
 'å',
 'v',
 'å',
 'n',
 'i',
 'ng',
 's',
 'h',
 'u',
 's',
 ',',
 '▁s'

## Design of the BPE Algorithm

Byte-pair encoding (BPE), due to Gage (1994), is the first algorithm to extract the subwords from a corpus. In the lab, you will first read two sections of more recent articles as they are easier to understand and specifically targeted to natural language processing.

Read these two sections:

1. Section 3.1 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 1 of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, in a *Method and program structure* section, you will introduce your program with a **summarization** (10 to 15 lines or so) with your own words of the byte-pair encoding (BPE) algorithm as described by Kudo (2018) and Bostrom and Durrett (2020) (Only BPE and not the unigram language model so far).

## BPE Programming

You will now program a byte-pair encoding program in Python. You will do it step by step. The first part will be to extract the subwords from a corpus. Note that you will use the characters, not the bytes.

Remember that your program should produce something similar in the end to the `m.vocab` file.

SentencePiece do not give a specific value to whitespace and replaces it with `▁` characters (U+2581).

Replace all the space sequences in `corpus`, including newlines and tabulations, and normalize them as one `▁` character (U+2581) https://www.unicode.org/charts/PDF/U2580.pdf. Use the `\s` class for this. When this is done, add a leading `▁` character to the resulting string.

In [111]:
# Write your code
# corpus = '▁' + re.sub(r'\s+', '▁', corpus_raw)
corpus = '\u2581' + re.sub(r'\s+', '\u2581', corpus_raw)

In [112]:
corpus[:100]

'▁Selma▁Lagerlöf▁En▁herrgårdssägen▁Bokutgåva▁Albert▁Bonniers▁förlag,▁Stockholm▁1899.▁I.▁Det▁var▁en▁sk'

### BPE

#### Initial Vocabulary

Write the code (one instruction) to split the corpus in a list of characters and store the results in `corpus_l`. This is just a type conversion. Given the input:

`corpus = '▁Selma▁Lagerlöf▁En▁herrgårdssägen▁Bokutgåva▁Albert...'`

Return:

`corpus_l = ['▁', 'S', 'e', 'l', 'm', 'a', '▁', 'L', 'a', 'g', 'e', 'r', 'l', 'ö', 'f', ...]`

In [113]:
corpus_l = [c for c in corpus]

In [114]:
corpus_l[:15]

['▁', 'S', 'e', 'l', 'm', 'a', '▁', 'L', 'a', 'g', 'e', 'r', 'l', 'ö', 'f']

Extract the set of characters that will serve as initial subword tokens: Write a statement to extract the set of all the characters from `corpus_l`

In [115]:
char_set = set(corpus_l)

In [116]:
sorted(char_set)

['!',
 ',',
 '-',
 '.',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'å',
 'é',
 'ö',
 '–',
 '’',
 '▁']

In [117]:
len(char_set)

68

Using the code from the previous question, write an `initial_vocabulary()` function taking the `corpus_l` variable as input and returning the set of all characters appearing in the corpus (the initial character set).

In [118]:
def initial_vocabulary(corpus_l):
    return set(corpus_l)

In [119]:
initial_vocabulary(corpus_l)

{'!',
 ',',
 '-',
 '.',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'å',
 'é',
 'ö',
 '–',
 '’',
 '▁'}

#### Counting

Write a `pair_count()` function that takes a list of tokens as input, single characters or subword tokens, and that counts the adjacent pairs (bigrams). You will implement these counts as dictionaries: The key will be a pair (tuple) of adjacent symbols and the value, its frequency. 

Note that you cannot cross whitespaces, here replaced by `▁` characters. i.e. a pair cannot include a `▁` except as a leading character.

A few examples:
  1. Legal subwords: `▁`, `▁a`, `▁ab`, `ab`;
  2. Illegal subwords:  `a▁`, `a▁b`.


Given the input

`['D', 'e', '▁', 's', 'e', 'n', 'a', 's', 't', ...]`
count_pairs should return a dictionary: 


`{('D', 'e'): 1, ('▁', 's'): 1, ('s', 'e'): 1, ('e', 'n'): 1, ('n', 'a'): 1, ...}`

In [120]:
def pair_count(corpus_l):
    pairs = {}
    for i in range(len(corpus_l)-1):
        c1 = corpus_l[i]
        c2 = corpus_l[i+1]
        #if c2 == '▁' or c1 == '.' or c1 == ',':
        if c2 == '▁':
            continue
        pairs[(c1, c2)] = pairs[(c1, c2)] + 1 if (c1, c2) in pairs else 1
    return pairs

In [121]:
pairs = pair_count(corpus_l)

In [122]:
Counter(pairs)

Counter({('▁', 's'): 4147,
         ('d', 'e'): 4016,
         ('▁', 'h'): 3974,
         ('e', 'n'): 3439,
         ('▁', 'd'): 2469,
         ('a', 'n'): 2441,
         ('t', 't'): 2178,
         ('a', 'r'): 1970,
         ('e', 't'): 1954,
         ('▁', 'a'): 1940,
         ('▁', 'v'): 1842,
         ('▁', 'f'): 1829,
         ('a', 't'): 1639,
         ('s', 't'): 1632,
         ('o', 'm'): 1610,
         ('▁', 'o'): 1603,
         ('o', 'n'): 1600,
         ('l', 'l'): 1551,
         ('▁', 'e'): 1519,
         ('a', 'd'): 1441,
         ('v', 'a'): 1426,
         ('h', 'a'): 1366,
         ('▁', 'm'): 1320,
         ('ö', 'r'): 1317,
         ('e', 'r'): 1281,
         ('c', 'h'): 1234,
         ('▁', 'b'): 1225,
         ('o', 'c'): 1223,
         ('▁', 'k'): 1200,
         ('▁', 't'): 1199,
         ('t', 'a'): 1180,
         ('i', 'g'): 1139,
         ('n', 'g'): 1119,
         ('t', 'e'): 1102,
         ('h', 'o'): 1098,
         ('r', 'a'): 1079,
         ('n', 'n'): 1070,
 

Determine the most frequent pair

In [123]:
# write your code
most_freq_pair = Counter(pairs).most_common(1)[0][0]
#most_freq_pair = max(pairs, key=lambda x: pairs[x])

In [124]:
most_freq_pair

('▁', 's')

In [125]:
''.join(most_freq_pair)

'▁s'

#### The First Iteration

We store the initial symbols in a `vocabulary` variable

In [126]:
vocabulary = initial_vocabulary(corpus_l)

In [127]:
len(vocabulary)

68

Add your most frequent pair to the vocabulary after one iteration as well as the merge operation in the `merge_ops` list. `merge_ops` will contain the merge operations in the order you created them.

In [128]:
merge_ops = []

In [129]:
merge_ops = []
merge_ops.append(most_freq_pair)
vocabulary.add(''.join(most_freq_pair))

In [130]:
len(vocabulary)

69

In [131]:
merge_ops

[('▁', 's')]

#### Incremental Construction
We will now incrementally build the vocabulary.

Create a `merge_bigrams()` function that takes a list of tokens, `corpus_l`, and a pair of subword tokens `(token_r, token_l)` as input and merges adjacent sequences token_r, token_l into a new token, `token_new`, replacing the sequence `token_r, token_l` in `corpus_l`. Your function will return a new list. 

Given the input 

`corpus_l = ['▁', 'D', 'e', '▁', 's', 'e', 'n', 'a', 's', 't', ...]`

`merge_bigrams(corpus_l, ('e', 'n'))` should return a list, where all the sequences of 'e' and 'n' have been merged:

`['▁', 'D', 'e', '▁', 's', 'en', 'a', 's', 't', ...]`

And reapplying `merge_bigrams(corpus_l, ('s', 'en'))` to this corpus should return

`['▁', 'D', 'e', '▁', 'sen', 'a', 's', 't', ...]`

You will apply a greedy algorithm. Given the pair ('a', 'a') and the list ['a', 'a', 'a'], the result will be: ['aa', 'a']

In [132]:
def merge_bigrams(corpus_l, pair):
    new_corpus = []
    i = 0
    while i < len(corpus_l)-1:
        if list(pair) == corpus_l[i:i+2]:
            new_corpus.append(''.join(pair))
            i += 2
        else:
            new_corpus.append(corpus_l[i])
            i += 1
    if i == len(corpus_l)-1:
        new_corpus.append(corpus_l[i])
    return new_corpus

In [133]:
corpus_test = ['▁', 'D', 'e', '▁', 's', 'e', 'n', 'a', 's', 't']
merge_bigrams(corpus_test, ('e', 'n'))

['▁', 'D', 'e', '▁', 's', 'en', 'a', 's', 't']

In [134]:
merge_bigrams(merge_bigrams(corpus_test, ('e', 'n')), ('s', 'en'))

['▁', 'D', 'e', '▁', 'sen', 'a', 's', 't']

#### Byte Pair Encoding (BPE): Building the Vocabulary

Write now a `BPE()` function following Algorithm 1 in _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ by Bostrom and Durrett (2020). 

Your function will take `corpus_l` and the vocabulary size `k` as input. This size `k` will correspond to the count of new subwords added to the initial list of symbols. 

You should have 68 found symbols in your initial corpus. With `k = 10`, you will add 10 subwords to this initial list. Note that Bostrom and Durrett (2020) define their $k_\text{Bostrom and Durrett}$ as `k + initial vocabulary`. 

Return the vocabulary of subword tokens in the form of a list, the initial vocabulary and the subwords you will create, as well as the merge operations in the order you created them

Your function will first extract the initial vocabulary and and then add `k` new symbols to this vocabulary.

In [135]:
# Write your code here
def BPE(corpus_l, k):
    vocabulary = initial_vocabulary(corpus_l) # set(corpus_l)
    merge_ops = []
    k = k + len(vocabulary)
    while len(vocabulary) < k:
        most_freq_pair = Counter(pair_count(corpus_l)).most_common(1)[0][0]
        merge_ops.append(most_freq_pair)
        vocabulary.add(''.join(most_freq_pair))
        corpus_l = merge_bigrams(corpus_l, most_freq_pair)
    return vocabulary, merge_ops

We build a vocabulary of 50 subwords in addition to our initial set of symbols

In [136]:
vocabulary, merge_ops = BPE(corpus_l, 50)

Check that you obtain the same sequences as in SentencePiece `m.vocab`

In [137]:
len(vocabulary), len(merge_ops)

(118, 50)

In [138]:
print(sorted(vocabulary))

['!', ',', ',▁s', '-', '.', '.▁H', '1', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'X', '_', 'a', 'ade', 'an', 'ar', 'b', 'c', 'ch', 'ck', 'd', 'de', 'e', 'en', 'er', 'et', 'f', 'g', 'h', 'i', 'ig', 'j', 'k', 'l', 'll', 'm', 'n', 'na', 'ng', 'o', 'om', 'on', 'or', 'p', 'r', 's', 'st', 't', 'tt', 'u', 'v', 'x', 'y', 'z', '»', 'Ä', 'Å', 'Ö', 'ä', 'än', 'är', 'å', 'é', 'ö', 'ör', '–', '’', '▁', '▁H', '▁a', '▁att', '▁b', '▁d', '▁de', '▁det', '▁e', '▁en', '▁f', '▁för', '▁g', '▁h', '▁han', '▁hon', '▁i', '▁k', '▁l', '▁m', '▁n', '▁o', '▁och', '▁p', '▁s', '▁t', '▁v', '▁var']


In [139]:
print(merge_ops)

[('▁', 's'), ('d', 'e'), ('▁', 'h'), ('e', 'n'), ('a', 'n'), ('t', 't'), ('a', 'r'), ('▁', 'v'), ('▁', 'f'), ('▁', 'a'), ('o', 'm'), ('o', 'n'), ('l', 'l'), ('▁', 'de'), ('▁', 'm'), ('ö', 'r'), ('▁', 'o'), ('c', 'h'), ('▁', 'b'), ('a', 'de'), ('▁', 'k'), ('▁', 't'), ('i', 'g'), ('▁a', 'tt'), ('e', 'r'), ('n', 'g'), ('▁o', 'ch'), ('s', 't'), ('▁', 'd'), ('▁h', 'on'), ('▁', 'g'), ('▁', 'i'), ('e', 't'), ('▁', 'e'), ('▁', 'l'), ('▁v', 'ar'), ('ä', 'r'), ('c', 'k'), ('▁f', 'ör'), ('▁', 'H'), ('▁', 'n'), ('▁h', 'an'), ('▁', 'p'), ('o', 'r'), ('n', 'a'), (',', '▁s'), ('ä', 'n'), ('▁', 'en'), ('.', '▁H'), ('▁de', 't')]


#### BPE Tokenizer

You will now use the vocabulary you obtained to tokenize a text stored in the corpus string.

You will apply the merge operations in the same order you created them. You will call this function `tokenize_bpe()` that will take two inputs: `corpus` and `merge_ops`, and that will return the tokenized text in the form of a list.

    def tokenize_bpe(corpus, merge_ops):

      ...

      return tokens

This function is easy. Just reuse the `merge_bigrams()` function and a loop

In [140]:
# Write your code here
def tokenize_bpe(corpus, merge_ops):
    corpus = [c for c in corpus]
    for op in merge_ops:
        corpus = merge_bigrams(corpus, op)
    return corpus

In [141]:
tokenize_bpe(['▁', 's'], merge_ops)

['▁s']

In [142]:
corpus

'▁Selma▁Lagerlöf▁En▁herrgårdssägen▁Bokutgåva▁Albert▁Bonniers▁förlag,▁Stockholm▁1899.▁I.▁Det▁var▁en▁skön▁höstdag▁hänemot▁slutet▁af▁trettiotalet.▁På▁den▁tiden▁fanns▁i▁Upsala▁ett▁högt,▁gult▁tvåvåningshus,▁som▁stod▁underligt▁ensamt▁på▁en▁liten▁äng,▁långt▁borta▁i▁en▁utkant▁af▁staden.▁Det▁var▁ett▁rätt▁ruskigt▁och▁otrefligt▁hus,▁men▁det▁förskönades▁af▁den▁massa▁vildvin,▁som▁växte▁där,▁och▁som▁på▁solsidan▁krälade▁så▁högt▁uppför▁den▁gula▁väggen,▁att▁det▁alldeles▁omramade▁de▁tre▁fönstren▁i▁öfvervåningen.▁I▁ett▁rum▁innanför▁ett▁af▁dessa▁inramade▁fönster▁satt▁en▁student▁och▁drack▁sitt▁morgonkaffe.▁Han▁var▁en▁lång,▁vacker▁karl▁med▁ett▁fint▁utseende.▁Håret▁bar▁han▁högt▁uppstruket▁ur▁pannan,▁det▁krusade▁sig▁vackert,▁och▁en▁lugg▁ville▁beständigt▁välta▁ned▁mot▁ögonen.▁Han▁var▁klädd▁i▁en▁bekväm▁och▁ledig▁dräkt,▁men▁var▁rätt▁elegant.▁Han▁hade▁det▁fint▁i▁sitt▁rum,▁där▁fanns▁en▁god▁soffa,▁och▁stoppade▁stolar,▁stort▁skrifbord▁och▁präktiga▁bokhyllor,▁men▁nästan▁inga▁böcker.▁Innan▁han▁hunnit▁dricka▁sitt▁kaffe

In [143]:
tokens_bpe = tokenize_bpe(corpus, merge_ops)
tokens_bpe

['▁',
 'S',
 'e',
 'l',
 'm',
 'a',
 '▁',
 'L',
 'a',
 'g',
 'er',
 'l',
 'ö',
 'f',
 '▁',
 'E',
 'n',
 '▁h',
 'er',
 'r',
 'g',
 'å',
 'r',
 'd',
 's',
 's',
 'ä',
 'g',
 'en',
 '▁',
 'B',
 'o',
 'k',
 'u',
 't',
 'g',
 'å',
 'v',
 'a',
 '▁',
 'A',
 'l',
 'b',
 'er',
 't',
 '▁',
 'B',
 'on',
 'n',
 'i',
 'er',
 's',
 '▁för',
 'l',
 'a',
 'g',
 ',',
 '▁',
 'S',
 't',
 'o',
 'ck',
 'h',
 'o',
 'l',
 'm',
 '▁',
 '1',
 '8',
 '9',
 '9',
 '.',
 '▁',
 'I',
 '.',
 '▁',
 'D',
 'et',
 '▁var',
 '▁en',
 '▁s',
 'k',
 'ö',
 'n',
 '▁h',
 'ö',
 'st',
 'd',
 'a',
 'g',
 '▁h',
 'än',
 'e',
 'm',
 'o',
 't',
 '▁s',
 'l',
 'u',
 't',
 'et',
 '▁a',
 'f',
 '▁t',
 'r',
 'e',
 'tt',
 'i',
 'o',
 't',
 'a',
 'l',
 'et',
 '.',
 '▁',
 'P',
 'å',
 '▁de',
 'n',
 '▁t',
 'i',
 'de',
 'n',
 '▁f',
 'an',
 'n',
 's',
 '▁i',
 '▁',
 'U',
 'p',
 's',
 'a',
 'l',
 'a',
 '▁e',
 'tt',
 '▁h',
 'ö',
 'g',
 't',
 ',',
 '▁g',
 'u',
 'l',
 't',
 '▁t',
 'v',
 'å',
 'v',
 'å',
 'n',
 'i',
 'ng',
 's',
 'h',
 'u',
 's',
 ',▁s',
 'o

### Comparison with SentencePiece
We now compare our results with those of SentencePiece

We tokenize the raw corpus

In [144]:
tokens_sp = sp.encode(corpus_raw, out_type=str)
tokens_sp

['▁',
 'S',
 'e',
 'l',
 'm',
 'a',
 '▁',
 'L',
 'a',
 'g',
 'er',
 'l',
 'ö',
 'f',
 '▁',
 'E',
 'n',
 '▁h',
 'er',
 'r',
 'g',
 'å',
 'r',
 'd',
 's',
 's',
 'ä',
 'g',
 'en',
 '▁',
 'B',
 'o',
 'k',
 'u',
 't',
 'g',
 'å',
 'v',
 'a',
 '▁',
 'A',
 'l',
 'b',
 'er',
 't',
 '▁',
 'B',
 'on',
 'n',
 'i',
 'er',
 's',
 '▁för',
 'l',
 'a',
 'g',
 ',',
 '▁',
 'S',
 't',
 'o',
 'ck',
 'h',
 'o',
 'l',
 'm',
 '▁',
 '1',
 '8',
 '9',
 '9',
 '.',
 '▁',
 'I',
 '.',
 '▁',
 'D',
 'et',
 '▁var',
 '▁en',
 '▁s',
 'k',
 'ö',
 'n',
 '▁h',
 'ö',
 'st',
 'd',
 'a',
 'g',
 '▁h',
 'än',
 'e',
 'm',
 'o',
 't',
 '▁s',
 'l',
 'u',
 't',
 'et',
 '▁a',
 'f',
 '▁t',
 'r',
 'e',
 'tt',
 'i',
 'o',
 't',
 'a',
 'l',
 'et',
 '.',
 '▁',
 'P',
 'å',
 '▁de',
 'n',
 '▁t',
 'i',
 'de',
 'n',
 '▁f',
 'an',
 'n',
 's',
 '▁i',
 '▁',
 'U',
 'p',
 's',
 'a',
 'l',
 'a',
 '▁e',
 'tt',
 '▁h',
 'ö',
 'g',
 't',
 ',',
 '▁g',
 'u',
 'l',
 't',
 '▁t',
 'v',
 'å',
 'v',
 'å',
 'n',
 'i',
 'ng',
 's',
 'h',
 'u',
 's',
 ',',
 '▁s'

And we compare the tokens. The lists should be identical

In [145]:
for i, (a, b) in enumerate(zip(tokens_sp, tokens_bpe)):
    if a != b:
        print(i, a, b)

158 , ,▁s
159 ▁s om
160 om ▁s
161 ▁s t
162 t o
163 o d
164 d ▁
165 ▁u u
198 ▁u ▁
199 t u
200 k t
201 an k
202 t an
203 ▁a t
204 f ▁a
205 ▁s f
206 t ▁s
207 ade t
208 n ade
209 . n
210 ▁ .
211 D ▁
212 et D
213 ▁var et
214 ▁e ▁var
215 tt ▁e
216 ▁ tt
217 r ▁
218 ä r
219 tt ä
220 ▁ tt
221 r ▁
222 u r
223 s u
224 k s
225 ig k
226 t ig
227 ▁och t
228 ▁o ▁och
229 t ▁o
230 r t
231 e r
232 f e
233 l f
234 ig l
235 t ig
236 ▁h t
237 u ▁h
238 s u
239 , s
240 ▁m ,
241 en ▁m
242 ▁det en
243 ▁för ▁det
244 s ▁för
245 k s
246 ö k
247 n ö
248 ade n
249 s ade
250 ▁a s
251 f ▁a
252 ▁de f
253 n ▁de
254 ▁m n
255 a ▁m
256 s a
258 a s
259 ▁v a
260 i ▁v
261 l i
262 d l
263 v d
264 i v
265 n i
266 , n
267 ▁s ,▁s
300 ▁u ▁
301 p u
303 f p
304 ör f
305 ▁de ör
306 n ▁de
307 ▁g n
308 u ▁g
309 l u
310 a l
311 ▁v a
312 ä ▁v
313 g ä
315 en g
316 , en
317 ▁att ,
318 ▁det ▁att
319 ▁a ▁det
320 ll ▁a
321 de ll
322 l de
323 e l
324 s e
325 ▁ s
326 om ▁
327 r om
328 a r
329 m a
330 ade m
331 ▁de ade
332 ▁t ▁de
333 r ▁t
334 e

## Unigram Language Model

You are now done with BPE and you can go to the unigram language model.

Read these two sections:

1. Section 3.2 of _Subword Regularization: Improving Neural Network Translation Models with Multiple Subword Candidates_ (https://arxiv.org/pdf/1804.10959.pdf) by Kudo (2018).
2. Section 2, algorithm 2 and the related text of _Byte Pair Encoding is Suboptimal for Language Model Pretraining_ (https://aclanthology.org/2020.findings-emnlp.414.pdf) by Bostrom and Durrett (2020).

In your report, in the *Method and program structure* section, you will introduce your program with **summarization** (10 to 15 lines or so) with your own words of the tokenization with a unigram language model as described by Kudo (2018) and Bostrom and Durrett (2020). You will notably consider two aspects:
1. How to obtain the subword vocabulary;
2. How to tokenize a text.

In your report, given what you have done on the byte-pair encoding, how would you build the “reasonably big seed vocabulary” needed for the unigram language model?

### Applying SentencePiece
We use the unigram model now

In [146]:
spm.SentencePieceTrainer.train(
    '--input=Selma/herrgard.txt --model_prefix=m --vocab_size=116 --user_defined_symbols=0,1,2,3,4,5,6,7,8,9')

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=Selma/herrgard.txt --model_prefix=m --vocab_size=116 --user_defined_symbols=0,1,2,3,4,5,6,7,8,9
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: Selma/herrgard.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 116
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: 0
  user_defined_symbols: 1
  user_defined_symbols: 2
  user_defined_symbols: 3
  user_defined_symbols: 4
  user_defined_symbols: 5
  user_defined_symbols: 6
  user_defined_sy

Once you have trained your model, look at the vocabulary and log-probabilities in `m.vocab`

In [147]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [148]:
sp.encode('senare', out_type=str)

['▁s', 'en', 'ar', 'e']

In [149]:
sp.encode('H', out_type=str)

['▁', 'H']

In [150]:
sp.encode('œ', out_type=str)

['▁', 'œ']

In [151]:
[sp.id_to_piece(i) for i in range(30)]

['<unk>',
 '<s>',
 '</s>',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '▁',
 't',
 'a',
 'r',
 'd',
 'g',
 'k',
 'e',
 'n',
 ',',
 'l',
 's',
 'en',
 '.',
 'm',
 'i',
 'o']

### Unigram Probabilities

Starting from the “reasonably big seed vocabulary”, you will now fit a unigram language model. You will start with a vocabulary of 50 subwords in addition to the character set and you will reduce it to 49, i.e. you will find one subword to discard.

Kudo (2018) proposes the expectation-maximization algorithm. We will ignore this step. Instead, in this lab, you will approximate the language model with the BPE algorithm.

Write a `unigram_lm()` function that takes a tokenized corpus as input and returns a dictionary, where the keys are the subwords and each key value, the key relative frequency:

    def unigram_lm(tokenized_corpus):

       ...

      return unigram_probs
You will:

1. Tokenize your corpus with BPE (we reuse the `tokenize_bpe()` function);
2. Write the function to estimate the probability of each word (simply count the occurrences of the subwords and divide them by the length of the tokenized corpus);
3. It is common practice to replace the probabilities with their logarithm. This will prevent multiplication underflow. Do this in your dictionary;
3. Return this model as a dictionary.

In [152]:
tokenized_corpus = tokenize_bpe(corpus, merge_ops)
tokenized_corpus[:15]

['▁', 'S', 'e', 'l', 'm', 'a', '▁', 'L', 'a', 'g', 'er', 'l', 'ö', 'f', '▁']

In [153]:
corpus

'▁Selma▁Lagerlöf▁En▁herrgårdssägen▁Bokutgåva▁Albert▁Bonniers▁förlag,▁Stockholm▁1899.▁I.▁Det▁var▁en▁skön▁höstdag▁hänemot▁slutet▁af▁trettiotalet.▁På▁den▁tiden▁fanns▁i▁Upsala▁ett▁högt,▁gult▁tvåvåningshus,▁som▁stod▁underligt▁ensamt▁på▁en▁liten▁äng,▁långt▁borta▁i▁en▁utkant▁af▁staden.▁Det▁var▁ett▁rätt▁ruskigt▁och▁otrefligt▁hus,▁men▁det▁förskönades▁af▁den▁massa▁vildvin,▁som▁växte▁där,▁och▁som▁på▁solsidan▁krälade▁så▁högt▁uppför▁den▁gula▁väggen,▁att▁det▁alldeles▁omramade▁de▁tre▁fönstren▁i▁öfvervåningen.▁I▁ett▁rum▁innanför▁ett▁af▁dessa▁inramade▁fönster▁satt▁en▁student▁och▁drack▁sitt▁morgonkaffe.▁Han▁var▁en▁lång,▁vacker▁karl▁med▁ett▁fint▁utseende.▁Håret▁bar▁han▁högt▁uppstruket▁ur▁pannan,▁det▁krusade▁sig▁vackert,▁och▁en▁lugg▁ville▁beständigt▁välta▁ned▁mot▁ögonen.▁Han▁var▁klädd▁i▁en▁bekväm▁och▁ledig▁dräkt,▁men▁var▁rätt▁elegant.▁Han▁hade▁det▁fint▁i▁sitt▁rum,▁där▁fanns▁en▁god▁soffa,▁och▁stoppade▁stolar,▁stort▁skrifbord▁och▁präktiga▁bokhyllor,▁men▁nästan▁inga▁böcker.▁Innan▁han▁hunnit▁dricka▁sitt▁kaffe

In [154]:
Counter(tokenized_corpus)

Counter({'▁': 5548,
         'a': 5390,
         't': 4266,
         'e': 3830,
         'r': 3629,
         'i': 3569,
         '▁s': 3546,
         'n': 3358,
         'å': 3265,
         'l': 2940,
         'd': 2863,
         'u': 2788,
         'en': 2299,
         's': 2229,
         '▁h': 2215,
         ',': 2141,
         'k': 2113,
         'g': 2007,
         'f': 1753,
         'an': 1704,
         '.': 1661,
         'om': 1610,
         'm': 1594,
         'j': 1558,
         'll': 1551,
         'ä': 1543,
         'o': 1479,
         'de': 1382,
         '▁m': 1320,
         '▁b': 1225,
         'ade': 1211,
         '▁k': 1200,
         'v': 1199,
         '▁t': 1199,
         'ar': 1147,
         'ig': 1139,
         '▁att': 1135,
         'er': 1096,
         'ng': 1086,
         '▁och': 1085,
         'st': 1072,
         '▁f': 1055,
         '▁d': 1046,
         'tt': 1043,
         '▁hon': 1022,
         'ö': 1019,
         '▁v': 1019,
         'p': 1018,
         

In [155]:
# Write your code here
def unigram_lm(tokenized_corpus):
    unigram_probs = {}
    total = sum(Counter(tokenized_corpus).values()) # total occurences
    for word, freq in Counter(tokenized_corpus).items():
        unigram_probs[word] = math.log(freq/total)
    return unigram_probs

In [156]:
unigram_logprobs = unigram_lm(tokenized_corpus)
sorted(unigram_logprobs.items(), key=lambda x: -x[1])

[('▁', -3.0574335453131423),
 ('a', -3.0863256628448217),
 ('t', -3.3201944277165687),
 ('e', -3.428006244573173),
 ('r', -3.481913919534794),
 ('i', -3.4985856032577045),
 ('▁s', -3.5050508401137113),
 ('n', -3.5595254891103787),
 ('å', -3.587611285037333),
 ('l', -3.6924614664151374),
 ('d', -3.7190010216497495),
 ('u', -3.7455465548674503),
 ('en', -3.938396801984683),
 ('s', -3.9693179933619955),
 ('▁h', -3.9756186442686285),
 (',', -4.009598038158182),
 ('k', -4.022762309131808),
 ('g', -4.074229977951527),
 ('f', -4.20954244182663),
 ('an', -4.237892619358604),
 ('.', -4.26345121713457),
 ('om', -4.294636868769356),
 ('m', -4.304624467397704),
 ('j', -4.327468100317171),
 ('ll', -4.331971163571326),
 ('ä', -4.337142474384704),
 ('o', -4.379504864037065),
 ('de', -4.447339322420249),
 ('▁m', -4.493239311167448),
 ('▁b', -4.567930203769038),
 ('ade', -4.579424583194772),
 ('▁k', -4.588549490971773),
 ('v', -4.589383171720351),
 ('▁t', -4.589383171720351),
 ('ar', -4.633721209618494

In [157]:
for token in vocabulary:
    if token not in unigram_logprobs:
        print(token)
        unigram_logprobs[token] = -1000

H


In [158]:
len(unigram_logprobs)

118

### Unigram Tokenization

You will now apply your unigram language model to tokenize a character sequence that does not include spaces, typically a single word in the Latin or Greek scripts or a sequence of words in Asian scripts, like Chinese or Korean.

To make it easier, use the `tokenize_lm()` function below that takes a character sequence, `char_seq`, and a dictionary of unigram log-probabilities, `unigram_logprobs`,  as input and returns the subword tokens and the segmentation log-probability, (prob, tokens). You will only return the token list with the highest log-probability.

    def tokenize_lm(char_seq, unigram_logprobs):

      ...

      return max(candidates)

As an example, applying 

`tokenize_lm('▁senare', unigram_logprobs)`
results in

`(-15.350338240623348, ['▁s', 'en', 'ar', 'e'])`

The function will cache (memoize) the results to speed up the computation. It is similar to that of Norvig's in the notebook: How to Do Things with Words.ipynb that you analyzed in the second lab.
Python has a built-in memoization function that you can use: @functools.lru_cache(maxsize=2**10). You can also use the newer @functools.cache() function if you have Python 3.9 or higher. See here: https://docs.python.org/3/library/functools.html.

In [159]:
import functools


def tokenize_lm(char_seq, unigram_probs):
    # Use one of the two cache functions below to have a faster answer:
    # @functools.lru_cache(maxsize=2**10)
    @functools.cache
    # Available from Python 3.9
    # The arguments of the cached function must be hashable that's why we define an inner cacheable function
    def __tokenize_lm(char_seq):
        if not char_seq:
            return 0.0, []
        splits = [(char_seq[:i], char_seq[i:])
                  for i in range(1, len(char_seq) + 1)]
        candidates = []
        # print(splits)
        for start, rest in splits:
            if start in unigram_probs:  # Else the probability is 0 as well as the product.
                start_prob = unigram_probs[start]
            else:
                start_prob = -1000
            rest_prob, rest_list = __tokenize_lm(rest)
            candidates.append(
                (start_prob + rest_prob, [start] + rest_list))
        # Uncomment the two next lines to see the recursion
        # print('\tCands.', candidates)
        # print('\t-> Max.:', max(candidates))
        return max(candidates)
    return __tokenize_lm(char_seq)

In [160]:
tokenize_lm('▁senare', unigram_logprobs)

(-15.50517509629006, ['▁s', 'en', 'ar', 'e'])

In [161]:
tokenize_lm('▁H', unigram_logprobs)

(-6.519571027533336, ['▁H'])

### Text Tokenization with Unigrams

The previous function applies to a sequence without spaces. You will now apply it to your corpus. Write a `tokenize_text_lm()` function that takes the whole `corpus` string as input and the unigram probabilities `unigram_probs` and return the corpus probability and the tokenized subwords. 

This function is just an application of the functions you just wrote, where you will:
1. Break the string tokens starting with the '▁' symbol. You may use `finditer()` and this regex: '▁[^▁]+'
2. Break the tokens into subtokens using `tokenize_lm()`. This function will return the probabilities of the resulting sequences;
3. Sum the logarithm of these probabilities.

It is very significant that you use the logarithm of the probabilities and the sum. If you multiply the probabilities, you will get an underflow.


In [162]:
re_token = '▁[^▁]+'

In [163]:
list(re.finditer(re_token, '▁kdlskdls▁ldösldös▁lödsldö'))

[<regex.Match object; span=(0, 9), match='▁kdlskdls'>,
 <regex.Match object; span=(9, 18), match='▁ldösldös'>,
 <regex.Match object; span=(18, 26), match='▁lödsldö'>]

In [164]:
# Write your code
def tokenize_text_lm(corpus, unigram_probs):
    tokenized_corpus = []
    corpus_prob = 0.0
    for token in re.finditer(re_token, corpus):
        prob, tokens = tokenize_lm(token.group(), unigram_probs)
        #print(token, prob, tokens)
        corpus_prob += prob
        tokenized_corpus += tokens
    return corpus_prob, tokenized_corpus

In [165]:
init_loglikelihood, tokens = tokenize_text_lm(
    corpus, unigram_logprobs)

In [166]:
init_loglikelihood, tokens[:10]

(-498811.42353027395, ['▁', 'S', 'e', 'l', 'm', 'a', '▁', 'L', 'a', 'g'])

### Vocabulary Selection

You will now implement the final loop, where you will:
1. Estimate the probabilities of the subwords with BPE and `unigram_lm()`

Then at each iteration:
1. Select one subword and remove it from your vocabulary.
3. Compute the resulting log-likelihood of the corpus without this word. You will use `tokenize_text_lm_2()` this time
4. Compute the loss, i.e. the log-likelihood reduction when the subword is removed from the current vocabulary

You will always keep the single characters in your vocabulary to avoid unknown words.

Store the pairs, (log-likelihood, removed_subword) in a list `logloss_word` and rank them by likelihood value.

In [167]:
vocabulary

{'!',
 ',',
 ',▁s',
 '-',
 '.',
 '.▁H',
 '1',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'R',
 'S',
 'T',
 'U',
 'V',
 'X',
 '_',
 'a',
 'ade',
 'an',
 'ar',
 'b',
 'c',
 'ch',
 'ck',
 'd',
 'de',
 'e',
 'en',
 'er',
 'et',
 'f',
 'g',
 'h',
 'i',
 'ig',
 'j',
 'k',
 'l',
 'll',
 'm',
 'n',
 'na',
 'ng',
 'o',
 'om',
 'on',
 'or',
 'p',
 'r',
 's',
 'st',
 't',
 'tt',
 'u',
 'v',
 'x',
 'y',
 'z',
 '»',
 'Ä',
 'Å',
 'Ö',
 'ä',
 'än',
 'är',
 'å',
 'é',
 'ö',
 'ör',
 '–',
 '’',
 '▁',
 '▁H',
 '▁a',
 '▁att',
 '▁b',
 '▁d',
 '▁de',
 '▁det',
 '▁e',
 '▁en',
 '▁f',
 '▁för',
 '▁g',
 '▁h',
 '▁han',
 '▁hon',
 '▁i',
 '▁k',
 '▁l',
 '▁m',
 '▁n',
 '▁o',
 '▁och',
 '▁p',
 '▁s',
 '▁t',
 '▁v',
 '▁var'}

In [168]:
# To have the same results
vocabulary_sorted = sorted(vocabulary, key=lambda w: -unigram_logprobs[w])
vocabulary_sorted[:10]

['▁', 'a', 't', 'e', 'r', 'i', '▁s', 'n', 'å', 'l']

In [169]:
corpus_l[0:2]

['▁', 'S']

In [170]:
len(merge_ops)

50

Initial unigram log-probabilities

In [171]:
tokenized_corpus = tokenize_bpe(corpus, merge_ops)
unigram_logprobs = unigram_lm(tokenized_corpus)
unigram_logprobs

{'▁': -3.0574335453131423,
 'S': -6.519571027533336,
 'e': -3.428006244573173,
 'l': -3.6924614664151374,
 'm': -4.304624467397704,
 'a': -3.0863256628448217,
 'L': -8.041040167021478,
 'g': -4.074229977951527,
 'er': -4.6792038592399035,
 'ö': -4.75204929352514,
 'f': -4.20954244182663,
 'E': -8.382789460743535,
 'n': -3.5595254891103787,
 '▁h': -3.9756186442686285,
 'r': -3.481913919534794,
 'å': -3.587611285037333,
 'd': -3.7190010216497495,
 's': -3.9693179933619955,
 'ä': -4.337142474384704,
 'en': -3.938396801984683,
 'B': -7.271907079483611,
 'o': -4.379504864037065,
 'k': -4.022762309131808,
 'u': -3.7455465548674503,
 't': -3.3201944277165687,
 'v': -4.589383171720351,
 'A': -7.535491600356332,
 'b': -5.971516061998989,
 'on': -5.319052458075487,
 'i': -3.4985856032577045,
 '▁för': -5.027054453158137,
 ',': -4.009598038158182,
 'ck': -5.001542865500729,
 'h': -6.202162774816354,
 '1': -11.678626326747864,
 '8': -11.678626326747864,
 '9': -10.985479146187918,
 '.': -4.263451217

In [172]:
def tokenize_text_lm_2(corpus, unigram_logprobs):
    tokenized_corpus = []
    corpus_prob = 0.0
    for word in re.finditer(re_token, corpus):
        prob, tokens = tokenize_lm(word.group(), unigram_logprobs)
        tokenized_corpus += tokens
        corpus_prob += prob
    return corpus_prob, tokenized_corpus

We run the expectation-maximization. We:
  1. Tokenize with BPE
  2. Estimate the subword probabilities
  3. Tokenize with unigram
  4. Estimate the subword probabilities
  5. Repeat steps 4. and 5.

until convergence to estimate the final probabilities

In [173]:
unigram_logprobs_c = unigram_logprobs.copy()
for i in tqdm.tqdm(range(5)):
    init_loglikelihood, tokens = tokenize_text_lm_2(corpus, unigram_logprobs_c)
    unigram_logprobs_c = unigram_lm(tokens)
    print(init_loglikelihood)

 20%|██        | 1/5 [00:01<00:06,  1.62s/it]

-498811.42353027395


 40%|████      | 2/5 [00:03<00:05,  1.69s/it]

-496764.3628773258


 60%|██████    | 3/5 [00:04<00:03,  1.61s/it]

-496582.118928692


 80%|████████  | 4/5 [00:06<00:01,  1.70s/it]

-496581.14903658227


100%|██████████| 5/5 [00:08<00:00,  1.68s/it]

-496581.14903658227


In [174]:
len(unigram_logprobs_c)

115

In [175]:
sorted(unigram_logprobs_c.items(), key=lambda x: -x[1])

[('▁', -3.0576706884963976),
 ('a', -3.0938400104723858),
 ('t', -3.2856455993023417),
 ('▁s', -3.3584091840700703),
 ('e', -3.459038525351995),
 ('i', -3.508508920826291),
 ('r', -3.513283540204391),
 ('å', -3.59753460260592),
 ('d', -3.656189496391951),
 ('l', -3.702384783983724),
 ('en', -3.73558285339332),
 ('n', -3.7465428358265944),
 ('u', -3.755469872436037),
 (',', -3.7721067841941918),
 ('.', -3.9747650277176967),
 ('s', -3.9792413109305826),
 ('▁h', -3.985541961837215),
 ('k', -4.0326856267003945),
 ('g', -4.092659726597914),
 ('f', -4.219465759395217),
 ('an', -4.251932379082225),
 ('om', -4.304560186337943),
 ('m', -4.314547784966291),
 ('j', -4.337391417885758),
 ('ll', -4.341894481139913),
 ('ä', -4.352263984295154),
 ('o', -4.3826896116324425),
 ('▁d', -4.447183360994134),
 ('▁m', -4.503162628736035),
 ('▁b', -4.577853521337624),
 ('ade', -4.589347900763359),
 ('▁k', -4.598472808540359),
 ('v', -4.599306489288937),
 ('▁t', -4.599306489288937),
 ('de', -4.627215277406013)

In [176]:
tokenize_lm('▁senare', unigram_logprobs_c)

(-15.196675090002465, ['▁s', 'en', 'ar', 'e'])

We remove one word at a time

In [177]:
logloss_word = []
for i, word in enumerate(tqdm.tqdm(vocabulary_sorted)):
    if len(word) == 1:
        continue
    # We store the word probability to be able restore it
    if word not in unigram_logprobs_c: #Added this to avoid error
        continue
    prob_word = unigram_logprobs_c.pop(word)
    loglikelihood, tokens = tokenize_text_lm_2(corpus, unigram_logprobs_c)
    log_loss = loglikelihood - init_loglikelihood
    logloss_word += [(log_loss, word)]
    # we restore the probability
    unigram_logprobs_c[word] = prob_word

  0%|          | 0/118 [00:00<?, ?it/s]

100%|██████████| 118/118 [01:20<00:00,  1.46it/s]


In [187]:
sorted(logloss_word, reverse=True)

[(-66.25974537036382, '▁o'),
 (-515.9344259195495, '▁a'),
 (-759.4485645494424, '▁de'),
 (-858.6955019260058, '▁en'),
 (-1098.3274613538524, 'na'),
 (-1212.3324421717552, '▁n'),
 (-1317.6505314345122, 'tt'),
 (-1361.352037640987, 'ch'),
 (-1466.2186934184865, '▁e'),
 (-1561.1524308869848, '▁l'),
 (-1565.9119940153323, 'ör'),
 (-1572.3923300793394, 'on'),
 (-1636.4594172228826, '▁i'),
 (-1655.8837524088449, 'än'),
 (-1857.4887331501232, 'et'),
 (-1974.4481175066321, '▁p'),
 (-2047.9354683588608, 'or'),
 (-2091.0677484131884, '▁t'),
 (-2094.115938058181, 'ar'),
 (-2262.2284465972916, '▁g'),
 (-2322.66477795213, '▁han'),
 (-2362.8821285502054, 'är'),
 (-2413.3725600343314, 'de'),
 (-2454.575197661412, '▁det'),
 (-2509.18603682745, 'er'),
 (-2690.126406208554, '▁f'),
 (-2737.4791472912766, 'st'),
 (-2798.3819719059393, '▁d'),
 (-2950.009653460642, '▁v'),
 (-2990.2602079851786, '▁k'),
 (-3031.255855624273, 'ng'),
 (-3289.723163012648, 'ig'),
 (-3645.9095006914577, '▁var'),
 (-3656.842298437

You will reduce now your vocabulary by one token: `out_candidate`. Write the piece of code to determine it.

In [179]:
# Write your code here
out_candidate = max(logloss_word)[1]

In [180]:
out_candidate

'▁o'

## The tokenization without the subword

In [181]:
vocabulary_sorted

['▁',
 'a',
 't',
 'e',
 'r',
 'i',
 '▁s',
 'n',
 'å',
 'l',
 'd',
 'u',
 'en',
 's',
 '▁h',
 ',',
 'k',
 'g',
 'f',
 'an',
 '.',
 'om',
 'm',
 'j',
 'll',
 'ä',
 'o',
 'de',
 '▁m',
 '▁b',
 'ade',
 '▁k',
 'v',
 '▁t',
 'ar',
 'ig',
 '▁att',
 'er',
 'ng',
 '▁och',
 'st',
 '▁f',
 '▁d',
 'tt',
 '▁hon',
 'ö',
 '▁v',
 'p',
 '▁g',
 '▁i',
 'et',
 '▁e',
 '▁l',
 '▁de',
 '▁var',
 'är',
 'ck',
 '▁för',
 '▁n',
 '▁han',
 'y',
 '▁p',
 'or',
 'na',
 ',▁s',
 '▁a',
 'än',
 '▁en',
 'on',
 '.▁H',
 '▁det',
 '–',
 'D',
 'ör',
 'I',
 'b',
 'M',
 'h',
 '▁o',
 'S',
 '▁H',
 'O',
 'ch',
 '_',
 '?',
 'J',
 'B',
 'G',
 'N',
 '!',
 'A',
 'F',
 'V',
 'Å',
 'x',
 'L',
 'K',
 'T',
 'E',
 'U',
 'P',
 'R',
 'Ä',
 ':',
 'é',
 'c',
 'Ö',
 '»',
 ';',
 'C',
 '9',
 'X',
 'z',
 '8',
 '’',
 '-',
 '1',
 'H']

In [182]:
vocabulary_sorted.remove(out_candidate)

In [183]:
unigram_probs_c = unigram_logprobs.copy()
unigram_probs_c.pop(out_candidate)
for i in range(5):
    new_loglikelihood, tokens = tokenize_text_lm_2(corpus, unigram_probs_c)
    unigram_probs_c = unigram_lm(tokens)
    print(new_loglikelihood)

-498924.4934099101
-496703.59818401857
-496521.35423540254
-496520.38434329757
-496520.38434329757


In [184]:
init_loglikelihood

-496581.14903658227

In [185]:
new_loglikelihood

-496520.38434329757

In [186]:
tokenize_text_lm(corpus, unigram_probs_c)

(-496520.38434329757,
 ['▁',
  'S',
  'e',
  'l',
  'm',
  'a',
  '▁',
  'L',
  'a',
  'g',
  'er',
  'l',
  'ö',
  'f',
  '▁',
  'E',
  'n',
  '▁h',
  'er',
  'r',
  'g',
  'å',
  'r',
  'd',
  's',
  's',
  'ä',
  'g',
  'en',
  '▁',
  'B',
  'o',
  'k',
  'u',
  't',
  'g',
  'å',
  'v',
  'a',
  '▁',
  'A',
  'l',
  'b',
  'er',
  't',
  '▁',
  'B',
  'on',
  'n',
  'i',
  'er',
  's',
  '▁för',
  'l',
  'a',
  'g',
  ',',
  '▁',
  'S',
  't',
  'o',
  'ck',
  'h',
  'o',
  'l',
  'm',
  '▁',
  '1',
  '8',
  '9',
  '9',
  '.',
  '▁',
  'I',
  '.',
  '▁',
  'D',
  'et',
  '▁var',
  '▁en',
  '▁s',
  'k',
  'ö',
  'n',
  '▁h',
  'ö',
  'st',
  'd',
  'a',
  'g',
  '▁h',
  'än',
  'e',
  'm',
  'o',
  't',
  '▁s',
  'l',
  'u',
  't',
  'et',
  '▁a',
  'f',
  '▁t',
  'r',
  'et',
  't',
  'i',
  'o',
  't',
  'a',
  'l',
  'et',
  '.',
  '▁',
  'P',
  'å',
  '▁d',
  'en',
  '▁t',
  'i',
  'd',
  'en',
  '▁f',
  'an',
  'n',
  's',
  '▁i',
  '▁',
  'U',
  'p',
  's',
  'a',
  'l',
  'a'

If you are interested, you can improve this program and test it on larger corpora. You can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will write a report where you will:
1. Write a short individual report on your program. I recommend that you use this structure for your report:
      1. Objectives and dataset
      2. Method and program structure, where you should outline your program and possibly describe difficult parts. You will have two subsections:
          * BPE
          * Unigram
      3. Results.
      4. Conclusion

1. Describe the background as well as the algorithms you used in the *Method and program structure* section. For this, summarize the articles as described in the notebook:
   * Design of the BPE Algorithm
   * Unigram Language Model

The whole report should be of 2 to 3 pages.

Submit your report as well as your **notebook** (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas,
1. Write directly your text in Canvas.

The submission deadline is October 4, 2024.

## Curious?

If you are interested, you can read the SentencePiece implementation by the author:
https://github.com/google/sentencepiece/tree/master

Concerning BPE, you can also read a fine implementation of BPE by Andrej Karpathy: https://github.com/karpathy/minGPT/blob/master/mingpt/bpe.py